# New Jersey Highlands Council Regional Review
## A breakdown of land preservation organizations and their holdings in the NJ Highlands Region
***
Johnathan Clementi <br>
Advanced Python Programming for GIS – PSU GEOG 489 <br>
Prof. James O’Brien, Grading Assistant Rossana Grzinic <br>
December 11, 2019 <br>
Final Project Deliverables 
***

### 1. Overview
This notebook creates a summary table of the New Jersey Highlands region municipalities with the acreage of preserved lands as well as the acreage of preservation region and planning region designations contained in those municipalities. The New Jersey Highlands Water Protection and Planning Council (often known as the Highlands Council is a regional planning agency tasked with implementing the Highlands Water Protection and Planning Act of 2004 (NJ Highlands Council, n.d.). In addition to the summary table, this notebook also creates two shapefiles and csv's for each municipality - one for the preserved lands data and one for the planning/preservation regions data. The main goal fo this project is to demonstrate my skills in GIS, python programming, multiprocessing, and pandas to achieve the production of a summary table.  

The project was built and tested in Python 3.6.9.

Python packages used:
* from the Python standard library:
    * os (https://docs.python.org/3/library/os.html)
    * sys (https://docs.python.org/3/library/sys.html)
    * multiprocessing (https://docs.python.org/3/library/multiprocessing.html)
    * time (https://docs.python.org/3/library/time.html)
    * re (https://docs.python.org/3/library/re.html)
    * zipfile (https://docs.python.org/3/library/zipfile.html)
* pandas (https://pypi.python.org/pypi/pandas)
* arcgis - ArcGIS API for Python (https://developers.arcgis.com/python/)

Resources:
1. New Jersey Highlands Council. https://www.nj.gov/njhighlands/

### 2. Preparation
To run this notebook, please make sure you have the requisite Python packages described above installed. Additionally, you will need to alter the input and output file paths in Section 2.2. 


An important note about multiprocessing in a Jupyter Notebook:
In order for multiprocessing to work in the Jupyter Notebook environment, it is necessary to have the worker function in a seperate python file located in the same directory as this notebook. The following resources document why multiprocessing needs a specific set up in order to be functional in Jupyter Notebooks (or other interactive shells). 

* Making Multiprocessing work in Jupyter Notebooks: https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac

* Why multiprocessing does not work in Jupyter or other interactive shells: https://stackoverflow.com/a/23641560/4613606

* Why multiprocessing does not work on Windows without the <code>if</code> clause: https://stackoverflow.com/questions/20222534/python-multiprocessing-on-windows-if-name-main

### 2.1 Importing packages and preparing notebook
The following code blocks contain the necessary package imports and variable assignments.

In [1]:
import os, sys
import arcpy
arcpy.env.overwriteOutput = True
import multiprocessing
import time
import re
import pandas as pd
import numpy as np

### 2.2 Data for this project
The data for this project was gathered from the New Jersey Highlands Council Open GIS data website (http://highlands-data-njhighlands.opendata.arcgis.com/). Please provide a *path* variable where the data will be located.

For the purposes of this project, the data has been supplied in the form of a geodatabase.

#### 2.2.1 Working Directory
Define the path of your working directory 

In [2]:
path = r'C:\Users\Johnathan\Google Drive\Grad School\PSU_GIS_Cert\GEOG 489\FinalPrj\clementi_FinalProject_Deliverables'

#### 2.2.2 New Jersey Municipalities Polygons
Data source: https://njogis-newjersey.opendata.arcgis.com/datasets/3d5d1db8a1b34b418c331f4ce1fd0fef_2

In [3]:
njMuni =  path + r'\data\HighlandsProtectedLands.gdb\NJ_Municipalities'

#### 2.2.3 New Jersey Highlands Region Boundary
Data source: http://highlands-data-njhighlands.opendata.arcgis.com/datasets/highlands-boundary

In [4]:
highlandsBoundary = path + r'\data\HighlandsProtectedLands.gdb\Highlands_Boundary'

#### 2.2.4 New Jersey Municipalities within the Highlands Region 
Data source: NJ_Municipalities clipped to Highlands_Boundary

In [5]:
highlandsMuni = path + r'\data\HighlandsProtectedLands.gdb\highlandsMuni'

#### 2.2.5 New Jersey Highlands Region Planning and Preservation Designation Polygons
Data source: http://highlands-data-njhighlands.opendata.arcgis.com/datasets/preservation-and-planning-area

In [6]:
planPresPoly = path + r'\data\HighlandsProtectedLands.gdb\Preservation_and_Planning_Area'

#### 2.2.6 New Jersey Highlands Region Preserved lands
Data source: http://highlands-data-njhighlands.opendata.arcgis.com/datasets/preserved-lands

In [7]:
presLands = path + r'\data\HighlandsProtectedLands.gdb\Preserved_Lands'

#### 2.2.7 Output directory
Initialize output directory name. Check if the output directory exists. Creates a directory if one does not exist

In [8]:
outFolder = path + r'\data\output'

if os.path.exists(outFolder):
    if os.path.isdir(outFolder):
        print('The proper output folder exists, moving on')
    else:
        os.mkdir(outFolder)
        print('Created the output directory')
else: 
    os.mkdir(outFolder)
    print('Created the output directory')

The proper output folder exists, moving on


#### 2.2.8 Output Table name
Initialize name for final result table

In [9]:
finalResult = path + r'\NJHC_SummaryTable.csv'

# Check if output exists, remove existing file
if os.path.exists(finalResult):
    os.remove(finalResult)
    print('Removed old output table')
else: 
    print('No output table to be removed')

Removed old output table


#### 2.2.9 Initializing Clip and Input features

In [10]:
# Define clip feature
clipper = highlandsMuni

# Define input features
tobeclipped = [presLands, planPresPoly]

### 3. Multiprocessing

### 3.1 Use 64 bit processing if available

In [11]:
def get_install_path():
    ''' Return 64bit python install path from registry (if installed and registered),
        otherwise fall back to current 32bit process install path.
    '''
    if sys.maxsize > 2**32: return sys.exec_prefix #We're running in a 64bit process
  
    #We're 32 bit so see if there's a 64bit install
    path = r'SOFTWARE\Python\PythonCore\2.7'
  
    from _winreg import OpenKey, QueryValue
    from _winreg import HKEY_LOCAL_MACHINE, KEY_READ, KEY_WOW64_64KEY
  
    try:
        with OpenKey(HKEY_LOCAL_MACHINE, path, 0, KEY_READ | KEY_WOW64_64KEY) as key:
            return QueryValue(key, "InstallPath").strip(os.sep) #We have a 64bit install, so return that.
    except: return sys.exec_prefix #No 64bit, so return 32bit path 

### 3.2 Multiprocessing Handler Function
Here we create the function that will handle all multiprocessing for this program. It creates a job list of each clip feature with the data that it will be passed (tobeclipped feature layers. Then these jobs are assigned to a multiprocessing worker which conducts the clip and produces outputs. Please consult comments within for how individual components function. 

In [12]:
def mp_handler():
     
    try:
        
        print("Creating Polygon OID list...") 
      
        # These are the fields we want to grab from the clip feature layer
        field = ['OID@', 'MUN_LABEL']
        
         # Create a list of object IDs for clipper polygons
        idList = []

        # Initialize list of municipality names (municipalities are used as clip features)
        clipperNameList = []

        # Iterate through the rows of the municipality feature layer (clipper) and return the OID and name field data
        with arcpy.da.SearchCursor(clipper, field) as cursor:
            for row in cursor:
                id = row[0] # Retrieve OID from first element in row 
                name = row[1] # Retrieve Municipality name from second element in row
                name = name.replace(" ", "_") # Replace illegal characters so we can use this field as the name of the output file later on
                name = name.replace("-", "_")
                idList.append(id)
                clipperNameList.append(name)
     
        print("There are " + str(len(idList)) + " object IDs (polygons) to process.") 


        # Reset field variable to just that of the OIDFieldName of the municipality feature layer
        clipperDescObj = arcpy.Describe(clipper) 
        field = clipperDescObj.OIDFieldName


        # Initialize tuples (not list because tuples are immutable) of tasks that will be sent to workers 
        jobs = []

        '''
            Nested loop creates job list for each input feature layer of clip (preserved lands and planning/preservation regions) and each feature of clip feature layer
            Use enumerate to get index of tobeclipped list then assign value at that index to a variable holding one element (instead of a list)
        '''
        for i, item in enumerate (tobeclipped):
            tobeclippeditem = tobeclipped[i] # Get just one clip input feature layer
            j = 0 # Initialize index used for retrieving municipality name 
            for id in idList:
                name = clipperNameList[j] # Get municipality name from current index
                j += 1 # Advance municipality name index
                jobs.append((clipper,tobeclippeditem,field,id,outFolder, name)) # Add tuples of the parameters that need to be given to the worker function to the jobs list

        print("Job list has " + str(len(jobs)) + " elements.") 


        ''' Multiprocessing Pool '''

        # Create and run multiprocessing pool.
        multiprocessing.set_executable(os.path.join(get_install_path(), 'pythonw.exe')) # make sure Python environment is used for running processes, even when this is run as a script tool

        print("Sending to pool") 

        cpuNum = multiprocessing.cpu_count()  # determine number of cores to use
        print("There are: " + str(cpuNum) + " cpu cores on this machine") 

        with multiprocessing.Pool(processes=cpuNum) as pool: # Create the pool object 
            res = pool.starmap(worker, jobs)  # run jobs in job list; res is a list with return values of the worker function


        ''' Error Reporting if successful try '''
 
        failed = res.count(False) # count how many times False appears in the list with the return values
        if failed > 0:
            arcpy.AddError("{} workers failed!".format(failed)) 
            print("{} workers failed!".format(failed)) 


        # If the process was completed, print a message 
        arcpy.AddMessage("Finished multiprocessing!") 
        print("Finished multiprocessing!")

        # Clean up in_memory
        arcpy.Delete_management("in_memory") 

        # Print processing time
        arcpy.AddMessage("Total time: %s seconds" % (time.time() - startTime))
    

    
    # Error Reporting if unsuccessful try            
    except arcpy.ExecuteError:
        # Geoprocessor threw an error 
        arcpy.AddError(arcpy.GetMessages(2)) 
        print("Execute Error:", arcpy.ExecuteError) 
    except Exception as e: 
        # Capture all other errors 
        arcpy.AddError(str(e)) 
        print("Exception:", e)


    # Clean up in_memory
    arcpy.Delete_management("in_memory") 

    # Print processing time
    arcpy.AddMessage("Total time: %s seconds" % (time.time() - startTime))

### 3.3 Run Multiprocessing

#### 3.3.1 Import worker Function
As noted in section 2, the worker function must live in a seperate python file located in the same directory as this Jupyter Notebook. Here is the worker code in a markdown cell.

```python 
import os, sys
import arcpy
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r'in_memory'
 
def worker(clipper, tobeclippeditem, field, oid, outFolder, name): 
    """  
       This is the function that does the work of clipping the input feature class to one of the polygons from the clipper feature class.  
       If the clip succeeds then it returns TRUE else FALSE.  
    """

    """ Prep """

    # Retrieve file location of current Feature Class - this is used to differentiate between Preserved Lands features and Planning/Preservation Region features
    tbcBseName = os.path.basename(tobeclippeditem)
  
    # Initialize a query string that will select the current municipality by OID 
    query = '"' + field +'" = ' + str(oid)

    # Initialize clipping municipality feature - use OID as a differentiating attribute (so workers dont use the same clippingMuni)
    clippingMuni = name + "clipper_" + str(oid)


    ''' 
        Create feature layer for a single municipality - this is used as the Clip feature in the clip operation 
        First parameter = (INPUT) feature class that holds clip features
        Second parameter = (OUTPUT) feature layer that is being created to hold single clip feature (in this case, one municipality)
        Third parameter = (INPUT) the query string created above - used to select municipality by its OID 
    '''
    arcpy.MakeFeatureLayer_management(clipper, clippingMuni, query) 
    
    # Initialize output feature layer name
    outFL = name + "_" + tbcBseName

    '''
        Conduct the clip
        First parameter = (INPUT) the current input layer (in this case either Preserved Lands or Planning / Preservation Regions)
        Second parameter = (INPUT) the current clip feature (refered to by the clipping municipality string created above)
        Third parameter = (OUTPUT) the output file being created 
    ''' 
    arcpy.Clip_analysis(tobeclippeditem, clippingMuni, outFL)


    # Create new fields for Municipality name and the area in acres
    arcpy.management.AddFields(outFL, [['MunName', 'TEXT', 200], ["Area", "FLOAT"]])

    # This is how to individually create fields
    #arcpy.AddField_management(outFL, "MunName", "TEXT")
    #arcpy.AddField_management(outFL, "Area", "FLOAT")

    # Add name of municipality to table
    arcpy.CalculateField_management(outFL, "MunName", '"' + name + '"')

    # Calculate the Acreage of the clip input polygons (preserved lands or ppRegions) in the new shapefile
    arcpy.CalculateField_management(outFL, "Area", "!shape.area@acres!")


    # Initialize name of output shapefile to go onto the disk
    outFC = os.path.join(outFolder, name + "_" + tbcBseName + ".shp")

    # Replacement for arcpy.env.overwriteOutput - check if output file exists, if it does, delete it
    if arcpy.Exists(outFC):
        arcpy.Delete_management(outFC)

    # Copy features from in memory to disk
    arcpy.CopyFeatures_management(outFL, outFC)
    
    # Initialize name for output csv file
    outCSV = name + "_" + tbcBseName + ".csv"

    # Check if csv exists, if it does, delete it, then recreate it
    if os.path.exists(outCSV):
        os.remove(outCSV)

    '''
        Write csv from attribute table for use in pandas
        First parameter = (INPUT) input table to be exported
        Second parameter = (OUTPUT) output path
        Third parameter = (OUTPUT) output file name
    '''
    arcpy.TableToTable_conversion(outFC, outFolder, outCSV)

    # Clean up in_memory
    arcpy.Delete_management(outFL) 
     
    arcpy.AddMessage("Finished clipping:" + str(oid)) 
    return True # everything went well so we return True
```

In [12]:
from workers import worker

#### 3.3.2 Start Multiprocessing
Here we start the multiprocessing script.

My machine with 12 logical processors and 16GB of RAM would complete this task in about 75-100 seconds.

In [14]:
arcpy.env.overwriteOutput = True

startTime = time.time()

if __name__ == '__main__':   
    mp_handler() 

Creating Polygon OID list...
There are 88 object IDs (polygons) to process.
Job list has 176 elements.
Sending to pool
There are: 12 cpu cores on this machine
Finished multiprocessing!


### 4. Summary Table Creation

### 4.1 Identifying tables for each municipality (both Preserved Lands and Planning/Preservation Regions)
There are 176 csv's (2 for each municipality in the Highlands region) in the output data folder. These files contain the caclulated acreage of the Preserved lands and Planning/Preservation Regions that fall within that municipality. Here we use regular expressions to create a list of preserved lands within municipalities and the planning/preservation regions within municipalities.

In [13]:
# Get list of files in output folder
pathFiles = os.listdir(outFolder)

#### 4.1.1 Identify tables that hold Preserved Lands data
Here we use a regular expression to create a list of all of the Preserved Lands csv's.

In [14]:
# Define regex expression for preserved lands files:
presPattern = '.*Preserved_Lands.csv$'
compiledPresRE = re.compile(presPattern)

# Create list of csv's matching regex expression
preservedLandFiles = []
for file in pathFiles:
    if compiledPresRE.match(file):
        preservedLandFiles.append(file)
preservedLandFiles
print(len(preservedLandFiles))

87


#### 4.1.2 Identify tables that hold Planning/Preservation Regions data
Here we use a regular expression to create a list of all of the Planning/Preservation Region csv's.

In [15]:
# Define regex expression for planning/preservation region files:
regionPattern = '.*Preservation_and_Planning_Area.csv$'
compiledPresRE = re.compile(regionPattern)

# Create list of csv's matching regex expression
ppRegionFiles = []
for file in pathFiles:
    if compiledPresRE.match(file):
        ppRegionFiles.append(file)
ppRegionFiles
print(len(ppRegionFiles))

87


#### Extra: Set pandas dataframe options for optimal display

In [18]:
# Show full table below
pd.options.display.max_rows = 10

# Change number of significant figures
pd.options.display.precision = 2

# Format floats to non-scientific method style
pd.options.display.float_format = '{:.2f}'.format

### 4.2 Reading data from csv's

#### 4.2.1 Read and concatenate Preserved Lands data
Using the list created in section 4.1.1, we read the data in the preserved lands csv into a dataframe then concatenate those dataframes into one large dataframe.

This section also includes formatting in the form of dropping unneeded columns. 

In [19]:
# Create list to hold dataframes
li = []

# Iterate through preserved lands files and load them into list of dataframes
for file in preservedLandFiles:
    df = pd.read_csv(outFolder + '\\' + file, sep=',', header = 0)
    li.append(df)

# Concatenate all preserved lands polygons into one dataframe
preservedDF = pd.concat(li, axis = 0)

# Remove unneeded columns
preservedDF = preservedDF.drop(['FID', 'OBJECTID','Shape_Leng', 'Acres'], axis=1)
preservedDF

,OS_CLASS,MunName,Area
0,MUNICIPAL,Alexandria_Township,1.66
1,FARMLAND,Alexandria_Township,9.74
2,FARMLAND,Alexandria_Township,78.74
3,FARMLAND,Alexandria_Township,119.57
4,FARMLAND,Alexandria_Township,1.78
...,...,...,...
168,MUNICIPAL,White_Township,1.83
169,STATE,White_Township,10.73
170,FARMLAND,White_Township,64.08
171,STATE,White_Township,3.95


#### 4.2.2 Read and concatenate Planning/Preservation Regions data
Using the list created in section 4.1.1, we read the data in the Planning/Preservation Regions csv into a dataframe then concatenate those dataframes into one large dataframe.

This section also includes formatting in the form of dropping unneeded columns. 

In [20]:
# Create list to hold dataframes
li = []

# Iterate through preserved lands files and load them into list of dataframes
for file in ppRegionFiles:
    df = pd.read_csv(outFolder + '\\' + file, sep=',', header = 0)
    li.append(df)

# Concatenate all preserved lands polygons into one dataframe
ppRegionsDF = pd.concat(li, axis = 0)

# Remove unneeded columns
ppRegionsDF = ppRegionsDF.drop(['FID', 'OBJECTID', 'GLOBALID', 'STATUS', 'ACRES','Shape_Leng'], axis=1)
ppRegionsDF

,REGION,MunName,Area
0,Highlands Planning Area,Alexandria_Township,15038.30
1,Highlands Preservation Area,Alexandria_Township,2721.25
0,Highlands Planning Area,Allamuchy_Township,7695.30
1,Highlands Preservation Area,Allamuchy_Township,5277.86
0,Highlands Planning Area,Alpha_Borough,1097.83
1,Highlands Preservation Area,Alpha_Borough,0.03
0,Highlands Planning Area,Bedminster_Township,15866.10
1,Highlands Preservation Area,Bedminster_Township,1009.36
0,Highlands Planning Area,Belvidere_Town,950.38
0,Highlands Planning Area,Bernardsville_Borough,8264.63


### 4.3 Summarizing data 

#### 4.3.1 Pivot and summarize Preserved Lands data
Here we pivot the Preserved lands dataframe so that we have the total acreage of each type of preservation organization broken down by each municipality. 

In [21]:
municipalPresDF = preservedDF.pivot_table(index='MunName', columns='OS_CLASS', values='Area',aggfunc=np.sum, fill_value=0)
# municipalPresDF.style.set_precision(2)
municipalPresDF

OS_CLASS,COUNTY,FARMLAND,FEDERAL,MUNICIPAL,NON-PROFIT,NONPROFIT,PRIVATE,STATE,TDR,WSMA
MunName,,,,,,,,,,
Alexandria_Township,312.81,2694.32,0.00,131.60,0.00,99.93,240.42,692.63,92.90,0.00
Allamuchy_Township,0.00,2401.59,0.00,514.79,0.00,79.24,175.01,5163.65,0.00,0.00
Alpha_Borough,0.00,133.47,0.00,45.21,0.00,0.00,0.00,0.00,0.00,0.00
Bedminster_Township,681.87,2197.62,0.00,932.05,0.00,134.66,459.96,18.73,39.46,0.00
Belvidere_Town,3.66,3.10,0.00,50.37,0.00,0.00,0.00,34.51,0.00,0.00
Bernards_Township,1296.18,36.73,2.28,1212.32,0.00,122.51,3.27,0.00,0.00,0.00
Bernardsville_Borough,108.40,0.00,175.81,183.98,0.00,264.81,52.02,0.00,0.00,0.00
Bethlehem_Township,781.22,1460.29,0.00,546.16,0.00,347.49,499.91,610.56,145.46,212.00
Bloomingdale_Borough,44.06,0.00,0.00,110.04,0.00,48.14,2.32,2337.05,0.00,197.10


Due to an issue with the data pulled from NJ Open GIS data, we combine the NON-PROFIT and NONPROFIT fields as they represent the same type organization. 

In [22]:
municipalPresDF.NONPROFIT = municipalPresDF.NONPROFIT+ municipalPresDF['NON-PROFIT']
municipalPresDF = municipalPresDF.drop(['NON-PROFIT'], axis=1)
municipalPresDF

OS_CLASS,COUNTY,FARMLAND,FEDERAL,MUNICIPAL,NONPROFIT,PRIVATE,STATE,TDR,WSMA
MunName,,,,,,,,,
Alexandria_Township,312.81,2694.32,0.00,131.60,99.93,240.42,692.63,92.90,0.00
Allamuchy_Township,0.00,2401.59,0.00,514.79,79.24,175.01,5163.65,0.00,0.00
Alpha_Borough,0.00,133.47,0.00,45.21,0.00,0.00,0.00,0.00,0.00
Bedminster_Township,681.87,2197.62,0.00,932.05,134.66,459.96,18.73,39.46,0.00
Belvidere_Town,3.66,3.10,0.00,50.37,0.00,0.00,34.51,0.00,0.00
Bernards_Township,1296.18,36.73,2.28,1212.32,122.51,3.27,0.00,0.00,0.00
Bernardsville_Borough,108.40,0.00,175.81,183.98,264.81,52.02,0.00,0.00,0.00
Bethlehem_Township,781.22,1460.29,0.00,546.16,347.49,499.91,610.56,145.46,212.00
Bloomingdale_Borough,44.06,0.00,0.00,110.04,48.14,2.32,2337.05,0.00,197.10


#### 4.3.2 Pivot and summarize Planning/Preservation Regions data
Here we pivot the Planning/Preservation Regions dataframe so that we have the total acreage of each type of each region type broken down by each municipality.

In [23]:
municipalppDF = ppRegionsDF.pivot_table(index='MunName', columns='REGION', values='Area', fill_value=0)
municipalppDF

REGION,Highlands Planning Area,Highlands Preservation Area
MunName,,
Alexandria_Township,15038.30,2721.25
Allamuchy_Township,7695.30,5277.86
Alpha_Borough,1097.83,0.03
Bedminster_Township,15866.10,1009.36
Belvidere_Town,950.38,0.00
Bernards_Township,15567.60,0.00
Bernardsville_Borough,8264.63,0.00
Bethlehem_Township,1141.31,12145.90
Bloomingdale_Borough,1762.31,4154.65


### 4.4 Joining Preserved Lands and Planning/Preservation Regions data
To create the final table, we join the two dataframes by using the municipality name as the primary keys of each table.

In [24]:
municipalDF = municipalppDF.merge(municipalPresDF, how='outer', left_on='MunName', right_on='MunName')
municipalDF

,Highlands Planning Area,Highlands Preservation Area,COUNTY,FARMLAND,FEDERAL,MUNICIPAL,NONPROFIT,PRIVATE,STATE,TDR,WSMA
MunName,,,,,,,,,,,
Alexandria_Township,15038.30,2721.25,312.81,2694.32,0.00,131.60,99.93,240.42,692.63,92.90,0.00
Allamuchy_Township,7695.30,5277.86,0.00,2401.59,0.00,514.79,79.24,175.01,5163.65,0.00,0.00
Alpha_Borough,1097.83,0.03,0.00,133.47,0.00,45.21,0.00,0.00,0.00,0.00,0.00
Bedminster_Township,15866.10,1009.36,681.87,2197.62,0.00,932.05,134.66,459.96,18.73,39.46,0.00
Belvidere_Town,950.38,0.00,3.66,3.10,0.00,50.37,0.00,0.00,34.51,0.00,0.00
Bernards_Township,15567.60,0.00,1296.18,36.73,2.28,1212.32,122.51,3.27,0.00,0.00,0.00
Bernardsville_Borough,8264.63,0.00,108.40,0.00,175.81,183.98,264.81,52.02,0.00,0.00,0.00
Bethlehem_Township,1141.31,12145.90,781.22,1460.29,0.00,546.16,347.49,499.91,610.56,145.46,212.00
Bloomingdale_Borough,1762.31,4154.65,44.06,0.00,0.00,110.04,48.14,2.32,2337.05,0.00,197.10


### 4.5 Prepare dataframe for export

#### 4.5.1 Reset Index
Until this point, the MunName field has been the index of our data. To ensure these data are exported, we reset the index.

In [25]:
finalDF = municipalDF.reset_index()
finalDF

,MunName,Highlands Planning Area,Highlands Preservation Area,COUNTY,FARMLAND,FEDERAL,MUNICIPAL,NONPROFIT,PRIVATE,STATE,TDR,WSMA
0,Alexandria_Township,15038.30,2721.25,312.81,2694.32,0.00,131.60,99.93,240.42,692.63,92.90,0.00
1,Allamuchy_Township,7695.30,5277.86,0.00,2401.59,0.00,514.79,79.24,175.01,5163.65,0.00,0.00
2,Alpha_Borough,1097.83,0.03,0.00,133.47,0.00,45.21,0.00,0.00,0.00,0.00,0.00
3,Bedminster_Township,15866.10,1009.36,681.87,2197.62,0.00,932.05,134.66,459.96,18.73,39.46,0.00
4,Belvidere_Town,950.38,0.00,3.66,3.10,0.00,50.37,0.00,0.00,34.51,0.00,0.00
5,Bernards_Township,15567.60,0.00,1296.18,36.73,2.28,1212.32,122.51,3.27,0.00,0.00,0.00
6,Bernardsville_Borough,8264.63,0.00,108.40,0.00,175.81,183.98,264.81,52.02,0.00,0.00,0.00
7,Bethlehem_Township,1141.31,12145.90,781.22,1460.29,0.00,546.16,347.49,499.91,610.56,145.46,212.00
8,Bloomingdale_Borough,1762.31,4154.65,44.06,0.00,0.00,110.04,48.14,2.32,2337.05,0.00,197.10
9,Bloomsbury_Borough,0.00,632.32,2.55,0.00,0.00,31.68,0.00,0.00,99.62,0.00,0.00


#### 4.5.2 Beautify table
Here we remove the underscores from the municipality names, replace N/A values with 0, and reformat the column names.

In [26]:
finalDF.MunName = finalDF.MunName.replace(to_replace="_", value=" ", regex=True)
finalDF = finalDF.fillna(value=0)
finalDF = finalDF.rename(columns={'MunName':'Municipality Name','Highlands Planning Area':'Planning Region','Highlands Preservation Area':'Preservation Region','COUNTY':'County','FARMLAND':'Farmland','FEDERAL':'Federal','MUNICIPAL':'Municipal','NONPROFIT':'Nonprofit','PRIVATE':'Private','STATE':'State'})
finalDF

,Municipality Name,Planning Region,Preservation Region,County,Farmland,Federal,Municipal,Nonprofit,Private,State,TDR,WSMA
0,Alexandria Township,15038.30,2721.25,312.81,2694.32,0.00,131.60,99.93,240.42,692.63,92.90,0.00
1,Allamuchy Township,7695.30,5277.86,0.00,2401.59,0.00,514.79,79.24,175.01,5163.65,0.00,0.00
2,Alpha Borough,1097.83,0.03,0.00,133.47,0.00,45.21,0.00,0.00,0.00,0.00,0.00
3,Bedminster Township,15866.10,1009.36,681.87,2197.62,0.00,932.05,134.66,459.96,18.73,39.46,0.00
4,Belvidere Town,950.38,0.00,3.66,3.10,0.00,50.37,0.00,0.00,34.51,0.00,0.00
5,Bernards Township,15567.60,0.00,1296.18,36.73,2.28,1212.32,122.51,3.27,0.00,0.00,0.00
6,Bernardsville Borough,8264.63,0.00,108.40,0.00,175.81,183.98,264.81,52.02,0.00,0.00,0.00
7,Bethlehem Township,1141.31,12145.90,781.22,1460.29,0.00,546.16,347.49,499.91,610.56,145.46,212.00
8,Bloomingdale Borough,1762.31,4154.65,44.06,0.00,0.00,110.04,48.14,2.32,2337.05,0.00,197.10
9,Bloomsbury Borough,0.00,632.32,2.55,0.00,0.00,31.68,0.00,0.00,99.62,0.00,0.00


### 4.6 Export final table

In [27]:
finalDF.to_csv(finalResult)